In [37]:
import pandas as pd
import numpy as np
import pathlib
from scipy.spatial.distance import jensenshannon
import sys

In [10]:
path_root_model = pathlib.Path(
    "/export/usuarios_ml4ds/lbartolome/Datasets/CORDIS/htm_variability_models/htm_6_tpcs_20230922")
path_submodel = pathlib.Path(
    "/export/usuarios_ml4ds/lbartolome/Datasets/CORDIS/htm_variability_models/htm_6_tpcs_20230922/submodel_htm-ws_from_topic_5_train_with_5_iter_0_20230922")

In [61]:
path_submodel.parent

PosixPath('/export/usuarios_ml4ds/lbartolome/Datasets/CORDIS/htm_variability_models/htm_6_tpcs_20230922')

In [8]:
sys.path.append('..')
from src.topicmodeler.src.topicmodeling.manageModels import TMmodel

In [11]:
tm_root = TMmodel(path_root_model.joinpath('TMmodel'))
tm_sub = TMmodel(path_submodel.joinpath('TMmodel'))

INFO:TMmodel:-- -- -- Topic model object (TMmodel) successfully created
INFO:TMmodel:-- -- -- Topic model object (TMmodel) successfully created


In [12]:
tm_root.get_tpc_word_descriptions(10)

[(0,
  'climate, climate_change, language, forest, biodiversity, archaeology, linguistic, ocean, ecological, heritage'),
 (1,
  'energy, robotic, aircraft, wireless, sensor, vehicle, energy_efficiency, hardware, automatic, engine'),
 (2,
  'quantum, molecule, photonic, energy, particle, optic, electron, laser, atom, galaxy'),
 (3,
  'cellular, genetic, protein, neuronal, molecule, genomic, brain, ribonucleic_acid, deoxyribonucleic_acid, vivo'),
 (4,
  'energy, water, carbon_dioxide, food, plant, chemical, waste, carbon, soil, metal'),
 (5,
  'patient, therapeutic, clinical, cancer, disease, tumor, drug, tissue, healthcare, biomarker')]

In [24]:
df,_ = tm_root.to_dataframe()
df.betas[0].shape

(6, 20254)

In [28]:
tm_root.get_tpc_word_descriptions(20254)[5][1].split(", ")[-1]

'control'

In [29]:
tm_sub.get_tpc_word_descriptions(10)

[(0,
  'health, healthcare, brain, pain, rehabilitation, alzheimer, patient, stroke, depression, parkinson'),
 (1,
  'tumor, cancer, cellular, drug, immunotherapy, therapeutic, metastasis, drug_delivery, chemotherapy, breast_cancer'),
 (2,
  'cardiac, cardiovascular, health, obesity, diabetes, cvd, heart, atrial_fibrillation, exposure, cardiovascular_disease'),
 (3,
  'image, imaging, nuclear_magnetic_resonance, diagnosis, biomarker, positron_emission, computed_tomography, cancer, point_care, detection'),
 (4,
  'vaccine, antibiotic, bone, infection, tuberculosis, biomaterial, cellular, pathogen, tissue, infectious_disease')]

In [32]:
def sim_word_comp(betas1: np.array,
                  betas2: np.array,
                  npairs: int,
                  thr: float = 1e-3):  # -> list(tuple(int, int, float)):
    """Calculates similarities between word distributions of two topic models based on their word composition using Jensen-Shannon distance.

    Parameters
    ----------
    betas1 : np.array
        Topic-word distribution of topic model 1, of dimensions (n_topics, n_words)
    betas2 : np.array
        Topic-word distribution of topic model 2, of dimensions (n_topics, n_words)
    npairs : int
        Number of pairs of words to be returned
    thr : float
        Threshold for removing words with low probability

    Returns
    -------
    selected_worddesc : list(tuple(int, int, float))
        List of tuples with the indices of the topics with the highest similarity and their similarity score
    """

    ntopics = betas1.shape[0]
    ntopics2 = betas2.shape[0]

    js_mat = np.zeros((ntopics, ntopics2))
    for k in range(ntopics):
        for kk in range(ntopics2):
            js_mat[k, kk] = jensenshannon(
                betas1[k, :], betas2[kk, :])
    JSsim = 1 - js_mat

    return JSsim

In [58]:
betas = tm_root.to_dataframe()[0].betas[0]
betas_sim = sim_word_comp(betas,betas, 2)
tpc_sim_from = 5
least_sim_tpc = np.argmin(betas_sim[tpc_sim_from,:])
# Most probable word of least similar topic
least_sim_word = tm_root.get_tpc_word_descriptions(betas.shape[1])[least_sim_tpc][1].split(", ")[0]
least_sim_word

'quantum'

In [128]:
n, p = 1, .5  # number of trials, probability of each trial
s = np.random.binomial(n, p, 1)[0]
s

0

For each submodel in root model;
    For each topic in submodel:
        Sample y from a Bernoulli to determine whether to introduce intruder or not.
        If y == 1:
            Sample z from a Categorical(1/3,1/3,1/3) to determine whether the intruder is going to be introduce in HTM-WS , HTM-DS or HLDA model
            Get most probable word of least similar topic as the intruder according to JS similarity on the betas
            If z == 0: 
                Insert the intruder in HTM-WS model
            If z == 1:
                Insert the intruder in HTM-DS model
            If z == 2:
                Get most similar topic in hlda model according to WMD
                Insert the intruder in hlda model's most similar topic
        If y == 0:
            Leave the topisc as they are
        

In [129]:
p_intruder_location = [1/3, 1/3, 1/3]  # Probability distribution for intruder location

In [195]:
np.random.choice(len(p_intruder_location), p=p_intruder_location)

1

In [198]:
 tm_root.get_tpc_word_descriptions()[0][1]

'climate, climate_change, language, forest, biodiversity, archaeology, linguistic, ocean, ecological, heritage, child, land, cognitive, sea, marine'

In [201]:
path_submodels = (
    pathlib.Path("/export/usuarios_ml4ds/lbartolome/Datasets/CORDIS/htm_variability_models/htm_6_tpcs_20230922/submodel_htm-ws_from_topic_2_train_with_10_iter_1_20230922"),
    pathlib.Path("/export/usuarios_ml4ds/lbartolome/Datasets/CORDIS/htm_variability_models/htm_6_tpcs_20230922/submodel_htm-ds_thr_0.6_from_topic_2_train_with_10_iter_1_20230930")
)

In [202]:
tms = {'parent': TMmodel(path_submodels[0].parent.joinpath('TMmodel')),
       'ws': TMmodel(path_submodels[0].joinpath('TMmodel')),
       'ds': TMmodel(path_submodels[1].joinpath('TMmodel'))
}

INFO:TMmodel:-- -- -- Topic model object (TMmodel) successfully created
INFO:TMmodel:-- -- -- Topic model object (TMmodel) successfully created
INFO:TMmodel:-- -- -- Topic model object (TMmodel) successfully created


In [223]:
topics1 = [el[1].split(', ') for el in tms['ws'].get_tpc_word_descriptions()]
topics2 = [el[1].split(', ') for el in tms['ds'].get_tpc_word_descriptions()]
topics0 = [el[1].split(', ') for el in tms['parent'].get_tpc_word_descriptions()]

In [200]:
import gensim.downloader as api
model = api.load('word2vec-google-news-300')

INFO:gensim.models.keyedvectors:loading projection weights from /home/lbartolome/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (3000000, 300) matrix of type float32 from /home/lbartolome/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz', 'binary': True, 'encoding': 'utf8', 'datetime': '2023-10-16T14:59:16.723437', 'gensim': '4.3.2', 'python': '3.10.9 (main, Jan 11 2023, 15:21:40) [GCC 11.2.0]', 'platform': 'Linux-5.15.0-78-generic-x86_64-with-glibc2.35', 'event': 'load_word2vec_format'}


In [212]:
def wmd(topics1, topics2, n_words=10):
    all_dist = np.zeros((len(topics1), len(topics2)))
    for idx1, tpc1 in enumerate(topics1):
        for idx2, tpc2 in enumerate(topics2):
            all_dist[idx1, idx2] = model.wmdistance(
                tpc1[:n_words], tpc2[:n_words])
    return all_dist

In [222]:
topics1

[['molecule',
  'nanoparticle',
  'self_assembly',
  'nanostructure',
  'polymer',
  'supramolecular',
  'chemical',
  'nanomaterial',
  'surface',
  'nanoscale',
  'single_molecule',
  'electric',
  'self_assemble',
  'biological',
  'structure'],
 ['imaging',
  'x_ray',
  'image',
  'microscopy',
  'sensor',
  'nuclear_magnetic_resonance',
  'optic',
  'resolution',
  'measurement',
  'detection',
  'probe',
  'sensitivity',
  'detector',
  'spectroscopy',
  'molecule'],
 ['fluid',
  'earth',
  'flow',
  'earthquake',
  'seismic',
  'deformation',
  'turbulence',
  'multiscale',
  'mantle',
  'mechanical',
  'microstructure',
  'wave',
  'rock',
  'fracture',
  'mechanic'],
 ['conjecture',
  'geometric',
  'equation',
  'graph',
  'algebraic',
  'string_theory',
  'algebra',
  'random',
  'mathematic',
  'topology',
  'invariant',
  'theorem',
  'computer_science',
  'symmetry',
  'nonlinear'],
 ['solar_cell',
  'electric',
  'semiconductor',
  'silicon',
  'graphene',
  'fabrication

In [225]:
all_dist

array([[1.26792323, 1.3031166 , 1.29967466, 1.32751347, 1.28047634,
        1.26406407, 1.25874425, 1.26126149, 1.29059986, 1.32558986],
       [1.27816297, 1.25233829, 1.22255968, 1.19077231, 1.22659481,
        1.26878842, 1.27047777, 1.26251871, 1.25194262, 1.08526937],
       [1.21253175, 0.90461443, 1.0827793 , 0.80430819, 0.96297898,
        1.08240279, 1.02121359, 1.23927682, 0.81299152, 0.9480273 ],
       [1.22155389, 1.19096912, 1.18886857, 1.17729338, 1.1862696 ,
        1.2782168 , 0.893347  , 1.2699434 , 0.96002708, 1.20346746],
       [1.31003956, 1.27890417, 1.20825464, 1.27697927, 1.23832159,
        1.27000387, 1.19806708, 1.23075505, 1.10686862, 1.2406454 ],
       [1.29431382, 1.29146147, 1.27081266, 1.26978005, 1.27330971,
        1.29759188, 1.21025875, 1.29387019, 1.2349819 , 1.28183319]])

In [224]:
all_dist = wmd(topics0,topics2)

INFO:gensim.models.keyedvectors:Removed 1 and 0 OOV words from document 1 and 2 (respectively).
INFO:gensim.corpora.dictionary:adding document #0 to Dictionary<0 unique tokens: []>
INFO:gensim.corpora.dictionary:built Dictionary<19 unique tokens: ['biodiversity', 'climate', 'climate_change', 'ecological', 'forest']...> from 2 documents (total 19 corpus positions)
INFO:gensim.utils:Dictionary lifecycle event {'msg': "built Dictionary<19 unique tokens: ['biodiversity', 'climate', 'climate_change', 'ecological', 'forest']...> from 2 documents (total 19 corpus positions)", 'datetime': '2023-10-16T16:48:35.369320', 'gensim': '4.3.2', 'python': '3.10.9 (main, Jan 11 2023, 15:21:40) [GCC 11.2.0]', 'platform': 'Linux-5.15.0-78-generic-x86_64-with-glibc2.35', 'event': 'created'}
INFO:gensim.models.keyedvectors:Removed 1 and 1 OOV words from document 1 and 2 (respectively).
INFO:gensim.corpora.dictionary:adding document #0 to Dictionary<0 unique tokens: []>
INFO:gensim.corpora.dictionary:built D

In [218]:
most_sim_tpc_wmd = np.argmin(all_dist[tpc_sim_from,:])
least_sim_tpc

3

In [220]:
topics1[tpc_sim_from]

['photonic',
 'optic',
 'laser',
 'nonlinear',
 'chip',
 'pulse',
 'metamaterial',
 'plasmonic',
 'wavelength',
 'waveguide',
 'frequency',
 'sensing',
 'source',
 'fiber',
 'frequency_comb']

In [221]:
topics2[least_sim_tpc]

['laser',
 'optic',
 'imaging',
 'source',
 'pulse',
 'photonic',
 'metamaterial',
 'ultrafast',
 'nonlinear',
 'detector',
 'beam',
 'image',
 'wavelength',
 'microscopy',
 'light']

In [226]:
def sample_bernoulli(n=1, p=.5):
    return np.random.binomial(n, p, 1)[0]

In [249]:
sample_bernoulli(p=.5)

0